<img src='images/cin-logo.png' style='width:600px;'>

# IF997- TÓPICOS AVANÇADOS EM SI 1
## Projeto Incognia

### Grupo: Alisson Diego, Guilherme Santiago, Pedro Sena

## Base de Dados

| coluna                               | tipo   | descrição                                                                                     |
|--------------------------------------|--------|-----------------------------------------------------------------------------------------------|
| id                                   | string | identificador do login                                                                        |
| account_id                           | string | identificador da conta                                                                        |
| device_id                            | string | identificador do dispositivo                                                                  |
| installation_id                      | string | identificador da instalação                                                                   |
| timestamp                            | long   | timestamp (unixtime - em ms)                                                                  |
| is_from_official_store               | bool   | se o app utilizado foi instalado através de uma loja oficial                                  |
| is_emulator                          | bool   | uso de emulador                                                                               |
| has_fake_location_app                | bool   | se o dispositivo possui aplicação de geração localizações falsas                              |
| has_fake_location_enable             | bool   | se o dispositivo habilitou geração de localizações falsas                                     |
| probable_root                        | bool   | provável root do sistema                                                                      |
| device_age_ms                        | long   | idade do dispositivo (em ms)                                                                  |
| max_installations_on_related_devices | int    | quantidade máxima de instalações dos dispositivos associados à conta                          |
| never_permited_location_on_account   | bool   | se o dispositivo nunca habilitou coleta de localizações, quando outros dispositivos o fizeram |
| boot_count                           | int    | quantidade de reinicializações do dispositivo                                                 |
| wallpaper_count                      | int    | quantidade de wallpapers do dispositivo                                                       |
| n_accounts                           | int    | número de contas acessadas pelo dispositivo                                                   |
| ato                                  | bool   | se login está associado a um evento de account takeover                                       |

In [1]:
import pandas as pd 
import numpy as np
import sys
!{sys.executable} -m pip uninstall pyarrow --yes
!{sys.executable} -m pip install pyarrow

  Using cached pyarrow-6.0.1-cp39-cp39-win_amd64.whl (15.5 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Acesso negado: 'C:\\Python39\\Lib\\site-packages\\pyarrow\\_compute.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
def parquetParaCsv(nomeArquivo):
    caminho = './dataset/{}{}'.format(nomeArquivo, ".parquet4")
    arquivo = pd.read_parquet(caminho, engine="pyarrow")
    nomeArquivo = nomeArquivo.split('.')
    #arquivo.to_csv('./dataset/{}.csv'.format(nomeArquivo[0]))
    return arquivo


def csvParaParquet(caminhoCsv, caminhoParquet):
    arquivo = pd.read_csv(caminhoCsv)
    #arquivo.to_parquet(caminhoParquet)
    return arquivo


In [3]:
dataframe = pd.read_parquet('./dataset/logins.parquet4')
dataframe.head()

,id,account_id,device_id,installation_id,timestamp,is_from_official_store,is_emulator,has_fake_location_app,has_fake_location_enabled,probable_root,device_age_ms,max_installations_on_related_devices,never_permitted_location_on_account,boot_count,wallpaper_count,n_accounts,ato
0,799740346458,YgXAhVgdDLsozwto,fGb574abkmDJTQ6M,Uv9PEQZEwsW8nLOj,1.621290e+12,1.0,0.0,0.0,0.0,0.0,1.407468e+09,1.0,0.0,129.0,28.0,1.0,0.0
1,799740346459,bO+gPxgfZG9sXtCb,Xll03k9l/OL2r6LM,pR7skP90RuM3oqhK,1.620309e+12,1.0,0.0,0.0,0.0,0.0,1.622747e+10,2.0,0.0,131.0,224.0,1.0,0.0
2,799740346460,9mEBGcdz+RLzRNB4,qoKjqPhxqyALMsQ5,hzKrODiR6F0jG4rH,1.625228e+12,1.0,0.0,0.0,0.0,0.0,2.865500e+10,2.0,0.0,179.0,6.0,1.0,0.0
3,799740346461,tghz/Mlz069esl03,5fqsAP5QUp1M3PLC,krWwUuXbmBEP9g38,1.624645e+12,1.0,0.0,0.0,0.0,0.0,2.484419e+10,3.0,0.0,38.0,11.0,1.0,0.0
4,799740346462,9mEBGcdz+RLzRNB4,qoKjqPhxqyALMsQ5,hzKrODiR6F0jG4rH,1.619907e+12,1.0,0.0,0.0,0.0,0.0,2.334158e+10,2.0,0.0,NaN,NaN,1.0,0.0


In [4]:
dataframe.isna().sum()

id                                           42
account_id                                   44
device_id                                    42
installation_id                              44
timestamp                                    41
is_from_official_store                       43
is_emulator                                  44
has_fake_location_app                        41
has_fake_location_enabled                    46
probable_root                                42
device_age_ms                                42
max_installations_on_related_devices         41
never_permitted_location_on_account          42
boot_count                              2027822
wallpaper_count                         2027851
n_accounts                               256726
ato                                          40
dtype: int64

In [36]:
#new_dataframe = dataframe.drop('Unnamed: 0', axis=1)
new_dataframe = dataframe

#int_columns = ['is_from_official_store', 'is_emulator', 'has_fake_location_app',
 #      'has_fake_location_enabled', 'probable_root',
  #     'max_installations_on_related_devices',
   #    'never_permitted_location_on_account', 'boot_count', 'wallpaper_count',
    #   'n_accounts', 'ato']

#new_dataframe[int_columns] = new_dataframe[int_columns].fillna(0).astype('int64')

#device_age_mean = new_dataframe['device_age_ms'].mean()
#new_dataframe['device_age_ms'].fillna(device_age_mean, inplace=True)

#timestamp_mode = new_dataframe['timestamp'].mode()[0]
#new_dataframe['timestamp'].fillna(timestamp_mode, inplace=True)

In [6]:
new_dataframe.isna().sum()

id                                           42
account_id                                   44
device_id                                    42
installation_id                              44
timestamp                                    41
is_from_official_store                       43
is_emulator                                  44
has_fake_location_app                        41
has_fake_location_enabled                    46
probable_root                                42
device_age_ms                                42
max_installations_on_related_devices         41
never_permitted_location_on_account          42
boot_count                              2027822
wallpaper_count                         2027851
n_accounts                               256726
ato                                          40
dtype: int64

In [38]:
import time
def limparBd (df):
    print("Tamanho inicial: " + str(len(df)))
    inicio = time.time()
    colunas = df.columns
    cont = 0
    indices = []
    dados = df.to_numpy()
    
    for linha in dados:
        if not linhaCorreta(linha):
            indices.append(cont)
        cont += 1
    dados = np.delete(dados, indices, axis=0)
    df = pd.DataFrame(dados, columns=colunas)
    print("Qtd linhas inválidas: " + str(len(indices)))
    print("Tamanho final: " + str(len(df)))    
    print("Demorou: " + str(time.time() - inicio) + " segundos")
    df.to_parquet("teste.parquet4")
    return df

def stringValida(valores):
    for valor in valores:
        if valor == "" or valor == None:
            return False
    return True

def booleanValido(valores):
    for valor in valores:
        if valor != 1 and valor != 0:
            return False
    return True

def linhaCorreta(linha):
    verificarStrings = [linha[0], linha[1], linha[2], linha[3]]
    if not stringValida(verificarStrings):
        return False
    
    timestamp = linha[4]
    if timestamp is None or type(timestamp) != float:
        print(linha)
        return False
    
    verificarBoolean = [linha[5], linha[6], linha[7], linha[8], linha[9], linha[12], linha[16]]
    if not booleanValido(verificarBoolean):
        return False
    
    return True


#null_ids_list = np.where(new_dataframe['id'].isnull())
#for id in null_ids_list:
  #new_dataframe = new_dataframe.drop(id)
#print(len(new_dataframe))
# ids_list = np.where(new_dataframe['installation_id'].isnull())
# for id in ids_list:
#   new_dataframe = new_dataframe.drop(id)

# for id in np.where(new_dataframe['account_id '].isnull()):
#   new_dataframe = new_dataframe.drop(id)

In [39]:
new_dataframe = limparBd(new_dataframe)

Tamanho inicial: 5418786
Qtd linhas inválidas: 329
Tamanho final: 5418457
Demorou: 31.046228647232056 segundos


In [40]:
from datetime import datetime
def adicionaData(df):
    colunas = df.columns
    dados = df.to_numpy()
    datas = []
    for linha in dados:
        timestamp = int(linha[4]/1000)                   
        dataHora = datetime.fromtimestamp(timestamp)          
        datas.append(dataHora)        
    df = pd.DataFrame(dados, columns=colunas) 
    df["date_time"] = datas 
    df.to_parquet("teste.parquet4")
    return df

In [41]:
new_dataframe = adicionaData(new_dataframe)
new_dataframe.head()

ValueError: cannot convert float NaN to integer